In [107]:
import sys
sys.path.append('../scripts')

import utils,analysis,test_features
import SCCML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report 



HIP_CER_df = pd.read_pickle('../data/HIP_CER.pkl')
ICC_df = pd.read_pickle('../data/ICC_rms.pkl')
SIMS_df = pd.read_pickle('../data/SIMS.pkl')
BIG_df = pd.read_pickle('../data/30k_data.pkl')

In [108]:
#split the dataset into training and test set
from sklearn.model_selection import train_test_split

X = ICC_df.drop(['type'], axis=1).copy()
y = ICC_df['type'].copy()

# Your original class labels
class_labels = y
# Initialize the LabelEncoder
label_encoder = LabelEncoder()
# Fit and transform the class labels
y = label_encoder.fit_transform(class_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=.20,
                                                    random_state=19)
print(X_train.shape, X_test.shape)

(1235, 511) (309, 511)


In [109]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=500,subsample=1)

# model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [110]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

0.7637540453074434
0.7621731478377473


## ANN 

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [95]:
# # Convert the training and testing sets to tensors
# X_train = torch.tensor(X_train.values, dtype=torch.float32)
# y_train = torch.tensor(y_train, dtype=torch.float32)
# X_test = torch.tensor(X_test.values, dtype=torch.float32)
# y_test = torch.tensor(y_test, dtype=torch.float32)

# # Reshape the labels
# y_train = y_train.reshape(-1, 1)
# y_test = y_test.reshape(-1, 1)

In [105]:
# Create an instance of the model
input_size = X_train.shape[1]  # Number of features in the input data
hidden_size = 50  # Number of units in the hidden layer
model = BinaryClassifier(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1500
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Test the model
predictions = model(X_test)
y_pred = predictions.round().detach().numpy()

Epoch 100/1500, Loss: 0.19870707392692566
Epoch 200/1500, Loss: 0.07125356793403625
Epoch 300/1500, Loss: 0.029238438233733177
Epoch 400/1500, Loss: 0.013319162651896477
Epoch 500/1500, Loss: 0.007204569410532713
Epoch 600/1500, Loss: 0.004399036522954702
Epoch 700/1500, Loss: 0.002958123106509447
Epoch 800/1500, Loss: 0.002114534378051758
Epoch 900/1500, Loss: 0.0015826198505237699
Epoch 1000/1500, Loss: 0.001230415073223412
Epoch 1100/1500, Loss: 0.0009830694179981947
Epoch 1200/1500, Loss: 0.0008031452889554203
Epoch 1300/1500, Loss: 0.0006661175866611302
Epoch 1400/1500, Loss: 0.0005610179505310953
Epoch 1500/1500, Loss: 0.00047854683361947536


In [106]:
print(accuracy_score(y_test, y_pred))

0.7443365695792881
